In [ ]:
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import os
import json
import matplotlib as plt
from ipywidgets import widgets
from ipywidgets import interact, interactive, fixed, interact_manual

 Un cas d'utilisation courant pour ces widgets est de les appliquer en tant que décorateurs et de les utiliser pour spécifier les arguments de fonction. Par exemple, si vous souhaitez récupérer des informations qui résident dans un élément d'un tableau, vous pouvez utiliser un IntSlider pour contrôler l'indice qui est passé dans cette fonction. Vous pouvez ensuite définir une logique dans cette fonction pour afficher votre image; si votre liste est une liste de chemins d'image, vous pouvez charger l'image et l'afficher via matplotlib. Ces widgets peuvent également être imbriqués, vous offrant un haut degré de flexibilité avec très peu d'effort.
 

In [4]:
file_path='nhl_data/20162017'
year=file_path[-8:-4]


In [ ]:

# Get the total number of regular season and playoff games
total_regular_season_games = len([filename for filename in sorted(os.listdir(file_path)) if filename.endswith('.json') and filename[-6:-4]=='02'])
total_playoff_games = len([filename for filename in sorted(os.listdir(file_path)) if filename.endswith('.json')and filename[-6:-4]=='03'])

# Create a dropdown widget to select the season
season_dropdown = widgets.Dropdown(
    options=['Regular Season', 'Playoffs'],
    description='Season:',
)

# Create an IntSlider for Game ID with a maximum value that depends on the selected season
game_id_slider = widgets.IntSlider(
    min=1,
    max=total_regular_season_games,
    description='Game ID:',
    continuous_update=False,
)

# Function to update the maximum value of the slider based on the selected season
def update_max_game_id(change):
    if season_dropdown.value == 'Regular Season':
        game_id_slider.max = total_regular_season_games
    else:
        game_id_slider.max = total_playoff_games

# Attach the update_max_game_id function to the season dropdown's observe method
season_dropdown.observe(update_max_game_id, 'value')

#load data
def load_data(game_ID, season):
    if season=='regular_season':
        season_code='02'
    else:
        season_code='03'
    file_path = f"{file_path}/{year}{season_code}{game_ID}.json"
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            return json.load(file)
    else:
        return None
    

# Function to display game events
def display_game_events(game_data):
    # Implement your code to display game events on the ice rink image here
        # Load the ice rink image
    ice_rink_image = Image(filename='ice_rink.png', format='png')

    # Create an ipywidget output widget to display the image
    output = widgets.Output()
    with output:
        display(ice_rink_image)

    # Display the image
    display(output)

    # Iterate through game events and display event coordinates
    events = game_data.get('events', [])
    for event in events:
        x_coord = event.get('x')
        y_coord = event.get('y')
        event_type = event.get('type')
        
        if x_coord is not None and y_coord is not None:
            # Display event coordinates on the ice rink image
            plt.figure(figsize=(8, 6))
            plt.imshow(plt.imread('ice_rink.png'))
            plt.scatter(x_coord, y_coord, marker='o', color='red', label=event_type)
            plt.title(f"Event: {event_type}")
            plt.legend()
            plt.show()
        else:
            # If no coordinates are available, print event data
            print(f"Event: {event_type}")
            print(json.dumps(event, indent=4))
    
            
# Function to update the display based on the selected season and game ID
def update_display(change):
    season = 'regular_season' if season_dropdown.value == 'Regular Season' else 'playoffs'
    game_id = game_id_slider.value
    game_data = load_data(game_id, season)
    if game_data:
        display_game_events(game_data)
    else:
        print(f"Game data not found for Game ID {game_id} in the {season}.")

# Attach the update_display function to the widgets' observe methods
season_dropdown.observe(update_display, 'value')
game_id_slider.observe(update_display, 'value')

# Display the widgets and initial output
display(season_dropdown, game_id_slider)
update_display(None)  # Display the initial output



